In [27]:
# 107303538_1111_hw9_1

In [28]:
import torch
import torch.nn as nn
import numpy as np

數據轉tensor，已知t_c 未知t_u

In [29]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0] # 攝氏溫度參數
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # 未知參數(待得知關係的)
t_c = torch.tensor(t_c) # 轉成tensor
t_u = torch.tensor(t_u)

model線性模型

In [30]:
def model(t_u, w, b): # 模型線性公式 (假設t_u和t_c為線性關係 想輸入t_u輸出t_p)
  return w * t_u + b # weight(權重)和b(偏移值)

loss function

In [31]:
def loss_fn(t_p, t_c): # 預測值為t_p
  squared_diffs = (t_p - t_c)**2 # 以拋物線來看誤差
  return squared_diffs.mean() # 每一次會有一組參數 全部下去平均 想要知道的為總loss

初始化 確定模型有在運作

In [32]:
w = torch.ones(())
b = torch.zeros(())
t_p = model(t_u, w, b)
t_p

tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])

處理train和validation的indice

分成train和validation的數據數目

In [33]:
# 分成train和validation的數據
n_samples = t_u.shape[0] # 共幾筆資料
n_val = int(0.2 * n_samples) # n_val 為驗證集 數量為sample的0.2倍
# int( )字符串或數字轉換為整數

隨機分成train和validation的indice

In [34]:
# 處理indice
shuffled_indices = torch.randperm(n_samples) # 把sample變成random 會有更好的學習效果
train_indices = shuffled_indices[:-n_val] # train的indice數 => 從前面取 取到倒數幾個(由驗證集的數量決定) 若驗證集為2則取到倒數-2個
val_indices = shuffled_indices[-n_val:] # val的indice數 => 從倒數後幾個取 取到這個list底
train_indices, val_indices

(tensor([ 8,  2, 10,  5,  3,  0,  7,  1,  6]), tensor([9, 4]))

處理train和validation的數據

In [35]:
train_t_u = t_u[train_indices] # c和u中要被train的
train_t_c = t_c[train_indices]
val_t_u = t_u[val_indices] # c和u中要被驗證val的
val_t_c = t_c[val_indices]
train_t_un = 0.1 * train_t_u # normalize讓效果變好
val_t_un = 0.1 * val_t_u

最佳化資料庫

In [36]:
import torch.optim as optim # 最佳化資料庫

設定最佳化器( params + 學習率 )

In [37]:
params = torch.tensor([1.0, 0.0], requires_grad=True) # 設定初始為w b 為1.0 0.0 開啟對params處理grad模式
learning_rate = 1e-2 # 設定學習率(SGD學習率可小 Adam學習率可較大)
optimizer = optim.SGD([params], lr=learning_rate) # 設定SGD為最佳化方法 
# 需要對optim.SGD( )設定params(需要訓練的參數)和學習率

training_loop ( 多train和val和optimizer )

In [38]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c): # 設定train_loop 多train和val和optimizer
  for epoch in range(1, n_epochs + 1): # n_epochs為跌代次數
    
    # train
    train_t_p = model(train_t_u, *params) # model  *params=w, b
    train_loss = loss_fn(train_t_p, train_t_c) # loss 

    # val
    with torch.no_grad(): # 關掉grad 因為在驗證階段不用考慮 且它會增加額外cost
      val_t_p = model(val_t_u, *params) # 對於驗證集做一樣的處理 (也可以把w b寫成 *params)
      val_loss = loss_fn(val_t_p, val_t_c)
      assert val_loss.requires_grad == False # (可以不加) 若跑val_loss的時候 發現requires_grad要為false 否則就跳出assert error訊息
    
    # 必要三步驟 (grad設為0，回推grad求解，最佳化更新)
    optimizer.zero_grad() # 把 optimizer 中的 grad 設定為 0 防止導數值grad累加
    train_loss.backward() # 將train_loss逆向反向傳播 並且得出每個train_loss的導數 即dloss_dw.mean(), dloss_db.mean() 存到每個*params
    optimizer.step() # step為更新回傳 跌代數據*params
    
    if epoch <= 3 or epoch % 500 == 0: # 只顯示跌代1 2 3 次和 後面每500
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}," # training loss 要寫數字的話把該變數{ }起來 前面記得要有f
         f" Validation loss {val_loss.item():.4f}") 
  return params # 回傳w b

訓練開始(最後回傳w b)

In [39]:
# 嘗試用3000次跌代訓練
training_loop(
  n_epochs = 3000,
  optimizer = optimizer,
  params = params,
  train_t_u = train_t_un, # 有經過normalize
  val_t_u = val_t_un,
  train_t_c = train_t_c,
  val_t_c = val_t_c)

Epoch 1, Training loss 89.6366, Validation loss 38.6392
Epoch 2, Training loss 44.9269, Validation loss 2.5454
Epoch 3, Training loss 37.1240, Validation loss 0.5562
Epoch 500, Training loss 6.6685, Validation loss 5.3562
Epoch 1000, Training loss 2.9650, Validation loss 6.1116
Epoch 1500, Training loss 2.4965, Validation loss 6.4033
Epoch 2000, Training loss 2.4372, Validation loss 6.5099
Epoch 2500, Training loss 2.4297, Validation loss 6.5482
Epoch 3000, Training loss 2.4288, Validation loss 6.5619


tensor([  5.4815, -17.4260], requires_grad=True)